# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [6]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [10]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    tour = []
    was = [0] * n
    cur = instance[0]
    tour.append(0)
    was[0] = 1
    for i in range(n - 1):
        mi = -1
        num = 0
        for ver in range(n):
            if (was[ver] == 1):
                continue
            dst = euclidean_distance(cur, instance[ver])
            if (mi == -1 or mi > dst):
                mi = dst
                num = ver
        tour.append(num)
        was[num] = 1
        cur = instance[num]
    return tour

In [21]:
def solve_tsp_nearest_insertion(instance):
    n = len(instance)
    tour = []
    was = [0] * n
    cur = instance[0]
    tour.append(0)
    was[0] = 1
    for i in range(n - 1):
        gdst = -1
        num = 0
        for ver in range(n):
            if (was[ver] == 1):
                continue
            mdst = -1
            for j in range(len(tour)):
                dst = euclidean_distance(instance[ver], instance[tour[j]])
                if (mdst == -1 or dst < mdst):
                    mdst = dst
            if (gdst == -1 or mdst < gdst):
                gdst = mdst
                num = ver
        md = -1
        pos = -1
        if (len(tour) == 1):
            was[num] = 1
            tour.append(num)
            continue
        for j in range(len(tour)):
            da = euclidean_distance(instance[tour[j]], instance[num])
            db = euclidean_distance(instance[num], instance[tour[(j + 1) % len(tour)]])
            dc = euclidean_distance(instance[tour[j]], instance[tour[(j + 1) % len(tour)]])
            if (md == -1 or md > da + db - dc):
                md = da + db - dc
                pos = j
        tour = tour[0:(pos + 1)] + [num] + tour[(pos + 1):len(tour)]
        was[num] = 1
    return tour

In [13]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [22]:
run_all()

Solving instance d198.tsp… done in 0.11 seconds with tour length 18620 using NN and in 3.9 seconds with tour length 18052 using NI
Solving instance d493.tsp… done in 0.31 seconds with tour length 43646 using NN and in 6.2e+01 seconds with tour length 41576 using NI
Solving instance d657.tsp… done in 0.61 seconds with tour length 62176 using NN and in 1.5e+02 seconds with tour length 60195 using NI
Solving instance d2103.tsp… done in 7.8 seconds with tour length 87468 using NN and in 4.1e+03 seconds with tour length 86027 using NI
Solving instance pr107.tsp… done in 0.016 seconds with tour length 46678 using NN and in 0.5 seconds with tour length 53211 using NI
Solving instance pr152.tsp… done in 0.016 seconds with tour length 85702 using NN and in 1.4 seconds with tour length 86914 using NI
Solving instance pr439.tsp… done in 0.25 seconds with tour length 131282 using NN and in 4.6e+01 seconds with tour length 132780 using NI


## Выводы
Как видно по результатам выполнения программы на предоставленных тестах, алгоритм Nearest Insertion находит лучший ответ в большем количестве случаев, но его ответ лучше лишь незначительно и работает он на порядок дольше, чем Nearest Neighbour.